In [1]:
#!/usr/bin/env python
# coding: utf-8

import json
from requests import get
from bs4 import BeautifulSoup

In [2]:
def extract_html(url):
    response = get(url, allow_redirects=False)
    html = response.text
    soup = BeautifulSoup(html, "lxml-xml")
    return(soup)

In [3]:
def add_to_json(data, title, summary, url):
    data['Article'].append({
        'Title': title,
        'Summary': summary,
        'Link': url
    })
    return(data['Article'])

In [4]:
def save_json(data, fname):
    with open('{}.json'.format(fname), 'w') as outfile:
        json.dump(data, outfile)

In [5]:
data = {}
data['Article'] = []

for i in range(10):
    
    soup = extract_html("https://news.un.org/en/news/region/middle-east?page={}".format(i))
    stories = soup.findAll("h1", {"class": "story-title"})
    
    for n in range(len(stories)):
        title = stories[n].text
        url = stories[n].find('a').attrs['href']
        soup = extract_html("https://news.un.org/"+stories[n].find('a').attrs['href'])
        div = soup.findAll("div", {"class": "field-item even"})
        summary = div[2].find('p').text
        add_to_json(data, title, summary, url)
        
save_json(data, 'results')

In [6]:
import pandas as pd    

df = pd.json_normalize(data['Article'])
df

,Title,Summary,Link
0,Libyan elections to take place in December 202...,National elections in Libya will take place on...,/en/story/2020/11/1077692
1,Yemen: Stricken oil tanker must be inspected t...,It is “vital” that technical experts be grant...,/en/story/2020/11/1077562
2,"Beirut blast aftermath: Healing underway, city...",Those affected by the devastating explosions t...,/en/story/2020/11/1077452
3,UN envoy urges Yemen’s warring parties to plac...,The plight of Yemenis demands “nothing less th...,/en/story/2020/11/1077422
4,UN agency for Palestine refugees runs out of m...,The UN Relief and Works Agency for Palestine R...,/en/story/2020/11/1077332
...,...,...,...
95,"Funding shortfall affecting critical water, sa...",Children in the heart of the world’s worst hum...,/en/story/2020/06/1066192
96,"Yemen: ‘Hanging on by a thread’, UN chief requ...",More than five years of conflict have left Yem...,/en/story/2020/06/1065292
97,FROM THE FIELD: COVID-19-related ‘tragedy’ unf...,A “tragedy” fueled by the spread of COVID-19 w...,/en/story/2020/06/1065202
98,"Life and death for Yemen’s women and girls, as...","In mid-May, just as the COVID-19 pandemic arri...",/en/story/2020/05/1065122
